# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7f3c362370>
├── 'a' --> tensor([[-1.0725, -0.9364, -1.3357],
│                   [ 1.1236,  1.2205,  0.6759]])
└── 'x' --> <FastTreeValue 0x7f7f3c3625b0>
    └── 'c' --> tensor([[ 1.0114,  0.6016, -2.1302, -0.8440],
                        [ 1.6805,  0.1583, -0.6914,  1.4604],
                        [-0.1779,  0.0637,  0.7113,  0.9602]])

In [4]:
t.a

tensor([[-1.0725, -0.9364, -1.3357],
        [ 1.1236,  1.2205,  0.6759]])

In [5]:
%timeit t.a

63.4 ns ± 0.0563 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7f3c362370>
├── 'a' --> tensor([[ 0.2605,  0.4563,  0.8250],
│                   [-0.2931,  2.0889,  0.2939]])
└── 'x' --> <FastTreeValue 0x7f7f3c3625b0>
    └── 'c' --> tensor([[ 1.0114,  0.6016, -2.1302, -0.8440],
                        [ 1.6805,  0.1583, -0.6914,  1.4604],
                        [-0.1779,  0.0637,  0.7113,  0.9602]])

In [7]:
%timeit t.a = new_value

72.4 ns ± 0.0625 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.0725, -0.9364, -1.3357],
               [ 1.1236,  1.2205,  0.6759]]),
    x: Batch(
           c: tensor([[ 1.0114,  0.6016, -2.1302, -0.8440],
                      [ 1.6805,  0.1583, -0.6914,  1.4604],
                      [-0.1779,  0.0637,  0.7113,  0.9602]]),
       ),
)

In [10]:
b.a

tensor([[-1.0725, -0.9364, -1.3357],
        [ 1.1236,  1.2205,  0.6759]])

In [11]:
%timeit b.a

58.5 ns ± 0.0501 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7404, -1.3821, -0.2305],
               [-0.8545, -0.9762, -2.1347]]),
    x: Batch(
           c: tensor([[ 1.0114,  0.6016, -2.1302, -0.8440],
                      [ 1.6805,  0.1583, -0.6914,  1.4604],
                      [-0.1779,  0.0637,  0.7113,  0.9602]]),
       ),
)

In [13]:
%timeit b.a = new_value

494 ns ± 0.251 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

814 ns ± 0.788 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 23 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

146 µs ± 379 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 1.03 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7e845fbdc0>
├── 'a' --> tensor([[[-1.0725, -0.9364, -1.3357],
│                    [ 1.1236,  1.2205,  0.6759]],
│           
│                   [[-1.0725, -0.9364, -1.3357],
│                    [ 1.1236,  1.2205,  0.6759]],
│           
│                   [[-1.0725, -0.9364, -1.3357],
│                    [ 1.1236,  1.2205,  0.6759]],
│           
│                   [[-1.0725, -0.9364, -1.3357],
│                    [ 1.1236,  1.2205,  0.6759]],
│           
│                   [[-1.0725, -0.9364, -1.3357],
│                    [ 1.1236,  1.2205,  0.6759]],
│           
│                   [[-1.0725, -0.9364, -1.3357],
│                    [ 1.1236,  1.2205,  0.6759]],
│           
│                   [[-1.0725, -0.9364, -1.3357],
│                    [ 1.1236,  1.2205,  0.6759]],
│           
│                   [[-1.0725, -0.9364, -1.3357],
│                    [ 1.1236,  1.2205,  0.6759]]])
└── 'x' --> <FastTreeValue 0x7f7e845fbc70>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.8 µs ± 109 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7f3c326580>
├── 'a' --> tensor([[-1.0725, -0.9364, -1.3357],
│                   [ 1.1236,  1.2205,  0.6759],
│                   [-1.0725, -0.9364, -1.3357],
│                   [ 1.1236,  1.2205,  0.6759],
│                   [-1.0725, -0.9364, -1.3357],
│                   [ 1.1236,  1.2205,  0.6759],
│                   [-1.0725, -0.9364, -1.3357],
│                   [ 1.1236,  1.2205,  0.6759],
│                   [-1.0725, -0.9364, -1.3357],
│                   [ 1.1236,  1.2205,  0.6759],
│                   [-1.0725, -0.9364, -1.3357],
│                   [ 1.1236,  1.2205,  0.6759],
│                   [-1.0725, -0.9364, -1.3357],
│                   [ 1.1236,  1.2205,  0.6759],
│                   [-1.0725, -0.9364, -1.3357],
│                   [ 1.1236,  1.2205,  0.6759]])
└── 'x' --> <FastTreeValue 0x7f7f108a8f70>
    └── 'c' --> tensor([[ 1.0114,  0.6016, -2.1302, -0.8440],
                        [ 1.6805,  0.1583, -0.6914,  1.4604],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 73.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.2 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.0114,  0.6016, -2.1302, -0.8440],
                       [ 1.6805,  0.1583, -0.6914,  1.4604],
                       [-0.1779,  0.0637,  0.7113,  0.9602]],
              
                      [[ 1.0114,  0.6016, -2.1302, -0.8440],
                       [ 1.6805,  0.1583, -0.6914,  1.4604],
                       [-0.1779,  0.0637,  0.7113,  0.9602]],
              
                      [[ 1.0114,  0.6016, -2.1302, -0.8440],
                       [ 1.6805,  0.1583, -0.6914,  1.4604],
                       [-0.1779,  0.0637,  0.7113,  0.9602]],
              
                      [[ 1.0114,  0.6016, -2.1302, -0.8440],
                       [ 1.6805,  0.1583, -0.6914,  1.4604],
                       [-0.1779,  0.0637,  0.7113,  0.9602]],
              
                      [[ 1.0114,  0.6016, -2.1302, -0.8440],
                       [ 1.6805,  0.1583, -0.6914,  1.4604],
                       [-0.1779,  0.0637,  0.7113,  0.9602]],

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 362 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.0114,  0.6016, -2.1302, -0.8440],
                      [ 1.6805,  0.1583, -0.6914,  1.4604],
                      [-0.1779,  0.0637,  0.7113,  0.9602],
                      [ 1.0114,  0.6016, -2.1302, -0.8440],
                      [ 1.6805,  0.1583, -0.6914,  1.4604],
                      [-0.1779,  0.0637,  0.7113,  0.9602],
                      [ 1.0114,  0.6016, -2.1302, -0.8440],
                      [ 1.6805,  0.1583, -0.6914,  1.4604],
                      [-0.1779,  0.0637,  0.7113,  0.9602],
                      [ 1.0114,  0.6016, -2.1302, -0.8440],
                      [ 1.6805,  0.1583, -0.6914,  1.4604],
                      [-0.1779,  0.0637,  0.7113,  0.9602],
                      [ 1.0114,  0.6016, -2.1302, -0.8440],
                      [ 1.6805,  0.1583, -0.6914,  1.4604],
                      [-0.1779,  0.0637,  0.7113,  0.9602],
                      [ 1.0114,  0.6016, -2.1302, -0.8440],
                   

In [28]:
%timeit Batch.cat(batches)

145 µs ± 548 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

324 µs ± 950 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
